In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import gensim

# import nltk
# from nltk.corpus import stopwords

# import keras
from keras.layers import Dense
from keras.models import Sequential
# from keras import optimizers

C:\Users\Aarush\Anaconda3\envs\tensorflow\lib\site-packages\gensim-3.0.1-py3.5-win-amd64.egg\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


# Model

In [23]:
def model_train(X_train, y_train, batch_size = 15, nb_epoch = 5 ,optimizer = 'adam', loss = 'categorical_crossentropy', activation='tanh', hidden_neurons = 30):
    model = Sequential()
    model.add(Dense(units = 100,input_dim=100, activation = activation))
    model.add(Dense(hidden_neurons, activation = activation))
    model.add(Dense(classes, activation = 'softmax'))
    model.compile(optimizer = optimizer, loss= loss, metrics = ['accuracy'])
    model.fit(X_train, y_train, batch_size = batch_size, nb_epoch = nb_epoch, verbose = 1)
    return model

In [3]:
def eval_model(X_test, y_test, model):
    scores = model.evaluate(X_test, y_test)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [4]:
"""Pass the dataset with data in the format: class:content"""
"""Text pre-processing: Removing links, special characters, and digits. df_column[1] is also converted into lower case"""
def preprocess_dataset(df, df_column):
    df[df_column[1]] = df[df_column[1]].str.replace('-', ' ')
    df[df_column[1]] = df[df_column[1]].str.replace('(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '')
    df[df_column[1]] = df[df_column[1]].str.replace('[^a-zA-Z0-9 \n]', ' ')
    df[df_column[1]] = df[df_column[1]].str.replace('\d+', '')
    df[df_column[1]] = df[df_column[1]].str.lower()
    return df

In [5]:
"""To label the sentiment classes using integers. Not to be used for the neural network"""
def to_categorical(df):
    df.sentiment = pd.Categorical(df.sentiment)
    df['class'] = df.sentiment.cat.codes
    return df['class']

In [6]:
"""Function returns the one-hot representation of the sentiment classes"""
def to_OneHot(df, df_columns):
    b = pd.get_dummies(df[df_column[0]], prefix="")
    list1 = list(b)
    OneHot = b[list1[0]]
    OneHot = np.column_stack(b[list1[i]] for i in range(len(list1)))
    print(len(list1))
    print(OneHot)
    return OneHot

In [7]:
#   """Labels can be either to_OneHot function return value or to_categorical function return value"""
# def split_train_test(df, df_column, labels, test_split = 0.2):
#     X_train, X_test, y_train, y_test = train_test_split(df[df_column[1]], labels, test_size = test_split, random_state = 10)
#     return X_train, X_test, y_train, y_test

# Testing

In [8]:
df = pd.read_csv('../Datasets/Twitter Sentiment Analysis/train_data.csv')
df_column = list(df)      #Names of the columns of the dataframe
classes = df[df_column[0]].unique().size #Number of distinct classes for the dataset. 13 for the given dataset
df = preprocess_dataset(df, df_column)
df.head()

,sentiment,content
0,empty,tiffanylue i know i was listenin to bad habi...
1,sadness,layin n bed with a headache ughhhh waitin o...
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,wants to hang out with friends soon
4,neutral,dannycastillo we want to trade with someone w...


In [9]:
labels = to_OneHot(df, df_column)

13
[[0 0 1 ..., 0 0 0]
 [0 0 0 ..., 1 0 0]
 [0 0 0 ..., 1 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [10]:
X_train, X_test, y_train, y_test = train_test_split(df[df_column[1]], labels, test_size = 0.2, random_state = 10)

In [11]:
X_train = X_train.to_frame().reset_index()

In [12]:
print(y_train.shape)
X_train.shape

(24000, 13)


(24000, 2)

In [13]:
text_list = [list(filter(None, row['content'].split(" "))) for i, row in X_train.iterrows()]

# labeled_text_list = [LabeledSentence(text_list[i], str(df['label'][i])) for i in range(len(df))]
# model = gensim.models.Word2Vec(text_list, min_count=1)
# model.wv['trade']

In [14]:
print(text_list[1])

['i', 'don', 't', 'wanna', 'work', 'a', 'hour', 'shift', 'today', 'i', 'd', 'rather', 'whisper', 'sweet', 'nothings', 'into', 'the', 'earpiece', 'of', 'a', 'lady', 'friend', 's', 'cellphone']


In [17]:
model = gensim.models.Word2Vec(text_list, min_count = 1, workers = 4, iter=5)

word_vectors = model.wv


del model

# for epoch in range(10):
#     model.train(labeled_text_list)
#     model.alpha -= 0.02
#     model.min_alpha = model.alpha
#     model.train(labeled_text_list)

In [18]:
word_vectors.vocab

{'drinking': <gensim.models.keyedvectors.Vocab at 0x28788f3dac8>,
 'mzthave': <gensim.models.keyedvectors.Vocab at 0x28788eea470>,
 'moth': <gensim.models.keyedvectors.Vocab at 0x28788f68128>,
 'typing': <gensim.models.keyedvectors.Vocab at 0x28788eb1b70>,
 'entertain': <gensim.models.keyedvectors.Vocab at 0x287fe267be0>,
 'withdraws': <gensim.models.keyedvectors.Vocab at 0x287fe2675f8>,
 'fuel': <gensim.models.keyedvectors.Vocab at 0x28788f30eb8>,
 'grind': <gensim.models.keyedvectors.Vocab at 0x28789577da0>,
 'kidz': <gensim.models.keyedvectors.Vocab at 0x28788f30fd0>,
 'girlusocrazy': <gensim.models.keyedvectors.Vocab at 0x2878935f7f0>,
 'jx': <gensim.models.keyedvectors.Vocab at 0x28789369780>,
 'tat': <gensim.models.keyedvectors.Vocab at 0x2878936c940>,
 'gratiss': <gensim.models.keyedvectors.Vocab at 0x2878935fa90>,
 'thebettersexdoc': <gensim.models.keyedvectors.Vocab at 0x2878a888e10>,
 'ilovewtcc': <gensim.models.keyedvectors.Vocab at 0x28789365b70>,
 'meganduffy': <gensim.mod

In [19]:
X = []
for i in range(len(text_list)):
    X.append(np.zeros(100))
    for j in range(len(text_list[i])):       
        X[i] += word_vectors[text_list[i][j]]
    if len(text_list[i]) != 0:
        np.asarray(X[i]/len(text_list[i]))
X = np.asarray(X)        

In [20]:
print(X.shape)
err = 0
for i in range(len(text_list)):
    if len(text_list[i])==0:
        print(i)   

(24000, 100)
3702
7361
12412


In [21]:
print(X.shape)
print(type(X))
print(y_train.shape)
print(type(y_train))

(24000, 100)
<class 'numpy.ndarray'>
(24000, 13)
<class 'numpy.ndarray'>


In [25]:
model = model_train(X, y_train)

C:\Users\Aarush\Anaconda3\envs\tensorflow\lib\site-packages\keras-2.0.8-py3.5.egg\keras\models.py:852: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/5
24000/24000 [==============================] - ETA: 0s - loss: 2.1060 - acc: 0.2663
Epoch 2/5
24000/24000 [==============================] - ETA: 0s - loss: 2.0911 - acc: 0.2695
Epoch 3/5
24000/24000 [==============================] - ETA: 0s - loss: 2.0886 - acc: 0.2705
Epoch 4/5
24000/24000 [==============================] - ETA: 0s - loss: 2.0866 - acc: 0.2718
Epoch 5/5
24000/24000 [==============================] - ETA: 0s - loss: 2.0867 - acc: 0.2730


In [26]:
X_test = X_test.to_frame()

In [27]:
type(X_test)

pandas.core.frame.DataFrame

In [28]:
test_list = [list(filter(None, row['content'].split(" "))) for i, row in X_test.iterrows()]


In [29]:
Y = []
for i in range(len(test_list)):
    Y.append(np.zeros(100))
    count = 0
    for j in range(len(test_list[i])):
        if(test_list[i][j] in word_vectors):
            Y[i] += word_vectors[test_list[i][j]]
            count = count +1
    if len(test_list[i]) != 0:
        np.asarray(Y[i]/count)
Y = np.asarray(Y)        

C:\Users\Aarush\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [30]:
y_test.shape

(6000, 13)

In [32]:
eval_model(Y, y_test, model)

6000/6000 [==============================] - ETA: 0s

acc: 29.38%
